# final lynda crawler

In [226]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from selenium import webdriver
import time



input_number = int(input('please enter number of course unit -> '))

option = webdriver.ChromeOptions()
option.add_argument('headless')


print('please wait ...')
#######################################################################################################################

driver = webdriver.Chrome("C:\\Users\\Lenovo\\Desktop\\chromedriver.exe" , options=option)
driver.get('https://origin-www.lynda.com/Web-training-tutorials/88-0.html')

#opening page base on course count aim #######################################################################################################################

print('clicking the show more buttun')
n = len(driver.find_elements_by_css_selector('#category-courses > li.card-cont '))
print(f'course count : {n}')
show_more = driver.find_element_by_css_selector('#category-courses > li.show-more-row > button')


time.sleep(10)


if input_number > n :
    show_more.click()

    while True :
        buttun1 = driver.find_element_by_css_selector('#category-courses > li.show-more-row')
        buttun2 = driver.find_element_by_css_selector('#category-courses > li:nth-of-type('+str(n+1)+')')

        if buttun1 != buttun2:
            n = len(driver.find_elements_by_css_selector('#category-courses > li.card-cont '))            
            print(f'course count : {n}')
            if n >= input_number or show_more.is_displayed == False :
                break
            else :
                buttun1.click()       

# information of first page #######################################################################################################################


# site_path = "https://origin-www.lynda.com/Web-training-tutorials/88-0.html"
# page = requests.get(site_path)
page = BeautifulSoup(driver.page_source , 'html.parser')

courses = page.select('#category-courses > li.card-cont ')

links = page.select('#category-courses > li > div > div > div.col-xs-8.col-sm-9.card-meta-data > a')[:input_number]
links = [l['href'] for l in links]

titles = page.select('#category-courses > li> div > div > div.col-xs-8.col-sm-9.card-meta-data > a > div.title-author-info > h3')[:input_number]
titles = [t.text for t in titles]

levels = page.select('#category-courses > li > div > div > div.col-xs-8.col-sm-9.card-meta-data > a > div.meta > span.meta-level')[:input_number]
levels = [l.text for l in levels]

short_description = page.select('#category-courses > li > div > div > div.col-xs-8.col-sm-9.card-meta-data > a > div.title-author-info > div')[:input_number]
short_description = [d.text for d in short_description]

duration = page.select('#category-courses > li > div > div > div.col-xs-8.col-sm-9.card-meta-data > a > div.meta > span.meta-duration')[:input_number]
duration = [d.text for d in duration]

release = page.select('#category-courses > li > div > div > div.col-xs-8.col-sm-9.card-meta-data > a > div.meta > span.meta-released > span')[:input_number]
release = [r.text for r in release]


# fully information #######################################################################################################################



MAIN_DESCRIPTION=[]
AUTHORES=[]
AUTHORES_IMAGES=[]
TAGS=[]
MAIN_IMAGES=[]


for link in tqdm(links) :
    try :
        course_page = requests.get(link)
    except :
        time.sleep(5)
        course_page = requests.get(link)
    course_page = BeautifulSoup(course_page.text , 'html.parser')

    main_description = course_page.select('#overview > div > div.col-sm-7.col-md-6.col-lg-8.course-description > div')
    MAIN_DESCRIPTION.append(main_description[0].find(text=True).strip())

    authors = course_page.select('#overview > div > div.col-sm-2.col-md-3.col-lg-2.course-meta > div > a > img > cite')
    AUTHORES.append(authors[0].text)

    authors_pic = course_page.select('#overview > div > div.col-sm-2.col-md-3.col-lg-2.course-meta > div > a > img')
    AUTHORES_IMAGES.append(authors_pic[0]['data-lazy-src'])


    tags = course_page.select('#course-tags > div')
    tags = tags[0].find_all('a')
    TAGS.append([t.text for t in tags])

    main_image = course_page.select('#banner-thumbnail > img')
    MAIN_IMAGES.append(main_image[0]['data-lazy-error-src'])


# dataframe #######################################################################################################################
print('dataframe structure ...')
data = {
                            'TITLE' : titles,
                            'LEVEL':levels,
                            'SHORT_DESCRIPTION' : short_description,
                            'DURATION' : duration,
                            'RELEASE' : release,
                            'LINKS' : links,
                            'MAIN_DESCRIPTION' : MAIN_DESCRIPTION,
                            'AUTHORES' : AUTHORES,
                            'AUTHORES_IMAGES' : AUTHORES_IMAGES,
                            'TAGS' : TAGS,
                            'MAIN_IMAGES' : MAIN_IMAGES,
}

dataframe = pd.DataFrame(data)
dataframe.to_csv("C:\\Users\\Lenovo\\Desktop\\lynda_df.csv" , index=None , encoding='utf-8')
print('dataframe saved')
#auther images ######################################################################################################################

print('downloading authors image ...')
author_info = list(set(zip(AUTHORES , AUTHORES_IMAGES)))
for author in tqdm(author_info) :
    try : 
        file = requests.get(author[1])
    except :
        time.sleep(5)
        file = requests.get(author[1])
    with open('C:\\Users\\Lenovo\\Desktop\\authors\\' + author[0] + '.jpg' , 'wb') as f :
        f.write(file.content)
        

#course images ######################################################################################################################
        
print('downloading courses image ...')
for i,img in enumerate(tqdm(MAIN_IMAGES)) :
    try :                        
        file = requests.get(img)
    except :
        time.sleep(5)       
        file = requests.get(img)
    with open('C:\\Users\\Lenovo\\Desktop\\courses\\' + str(i) + '.jpg' , 'wb') as f :
        f.write(file.content)

please enter number of course unit -> 5
please wait ...
clicking the show more buttun
course count : 50


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

dataframe structure ...
dataframe saved
downloading authors image ...


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

downloading courses image ...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:14<00:00,  3.00s/it]
